<a href="https://colab.research.google.com/github/blusewill/ytvideo-decipher/blob/dev/ytvideo_decipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-decipher

The Project that is kinda fork to the [ytvideo-whisper](https://github.com/blusewill/ytvideo-whisper)

Uses the [Decipher Project](https://github.com/dsymbol/decipher)

## Add Google Drive Access 
(at Google Drive/Decipher)

In [ ]:
from google.colab import drive
import os

path = '/content/gdrive/MyDrive/decipher'
drive.mount('/content/gdrive')

if not os.path.exists(path):
  os.mkdir(path)

Mounted at /content/gdrive


## Install Decipher

In [ ]:
! nvidia-smi
! apt update && apt install ffmpeg fonts-open-sans
! pip install git+https://github.com/dsymbol/decipher
! pip install yt-dlp

Sun May 21 16:03:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Settings

In [ ]:
#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=yxf2VRuAtls" #@param {type:"string"}
# File Name Change
new_filename = "Conc\xE9deme Un Momento" #@param {type:"string"}
# Audio Language
#@markdown # **Transcribe Options**
language = "" #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
#@markdown # **Translate Options**
translate = "no" #@param ['yes','no']
translate_to_language = "english" #@param {type:"string"}
# Change the Model of the Whisper
#@markdown # **Other Options**
shutdown_after_complete = "yes" #@param ['yes','no']
model = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']

# Download the YouTube Video using yt-dlp

In [ ]:
import yt_dlp

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'aac',
        'preferredquality': '192',
    }],
        'outtmpl': 'content/audio',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=yxf2VRuAtls
[youtube] yxf2VRuAtls: Downloading webpage
[youtube] yxf2VRuAtls: Downloading android player API JSON
[youtube] yxf2VRuAtls: Downloading player e50626d8
[info] yxf2VRuAtls: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: content/audio
[download] 100% of    3.33MiB in 00:00:00 at 18.01MiB/s              
[ExtractAudio] Destination: content/audio.m4a
Deleting original file content/audio (pass -k to keep)


# Transcribe/Translate the Video

In [ ]:
if translate.lower() == "no":

  input = "content/audio.m4a"
  output_dir = "/content/gdrive/MyDrive/decipher/result"
  task = "transcribe"
  subtitle_action = "None"

  from decipher.action import transcribe
  import os

  transcribe(
    input, 
    output_dir,
    model,
    language if language else None,
    task,
    subtitle_action if subtitle_action != "None" else None
    )
  
elif translate.lower() == "yes":

  input = "content/audio.m4a"
  output_dir = "/content/gdrive/MyDrive/decipher/result"
  language = ""
  task = "translate"
  subtitle_action = "None"

  from decipher.action import transcribe
  import os
  transcribe(
    input, 
    output_dir,
    model,
    translate_to_language if translate_to_language else None,
    task,
    subtitle_action if subtitle_action != "None" else None
  )
  
else:
  print("There is no Selected Options. Shutting down the Runtime")
  from google.colab import runtime
  runtime.unassign()


Converting audio.m4a to audio.aac...


audio.m4a: 100%|██████████| 201/201 [00:00<00:00, 420266.75 seconds/s]
100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 133MiB/s]


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Spanish
[00:00.000 --> 00:22.160]  No sé si fue real o si fue una ilusión
[00:22.160 --> 00:30.640]  Sintado en aquel bar mientras yo al cantar
[00:30.640 --> 00:39.120]  Notaba que mis besos alcanzarán a tu corazón
[00:39.120 --> 00:47.920]  Soñando que cada mirada no fuera por casualidad
[00:47.920 --> 00:56.320]  De nuevo estoy aquí con mi vieja canción
[00:56.320 --> 01:06.320]  Sonríes para mí, este mismo y con el tiempo
[01:06.320 --> 01:13.680]  Amar chitado nuestra piel y nuestra juventud
[01:13.680 --> 01:22.320]  Pero conservo la magia del amor
[01:22.320 --> 01:32.160]  Tenerte cerca, amor, me da tranquilidad
[01:32.160 --> 01:41.200]  Anhelo tu calor, transmites tanta paz
[01:41.440 --> 01:50.560]  Siempre estás junto a mí, que debo ser para ti
[01:50.560 --> 02:00.240]  Quizás el hombre que te hace vivir en un sueño
[02:00.240 --> 02:09.040]  Te abrazo para oír 

# Rename the srt file

In [ ]:
current_file_path = '/content/gdrive/MyDrive/decipher/result/audio.srt'
directory_path = os.path.dirname(current_file_path)
file_name = os.path.basename(current_file_path)
new_file_path = os.path.join(directory_path, new_filename)
os.rename(current_file_path, new_file_path)

# Auto Shutdown the Runtime

In [ ]:
if shutdown_after_complete.lower() == "yes":
    from google.colab import runtime
    runtime.unassign()